In [17]:
# %pip install maxim-py

In [18]:
from dotenv import load_dotenv
import os
load_dotenv()

print(os.environ["LOG_REPO_ID"])

cm1ac3n9n0002fba0c58h1dx2


In [19]:
from maxim.maxim import Logger, LoggerConfig
from maxim.logger.components.session import Session, SessionConfig
from maxim.logger.components.span import Span, SpanConfig
from maxim.logger.components.trace import Trace, TraceConfig
from uuid import uuid4

base_url = "https://app.getmaxim.ai"
maxim_api_key = os.environ["MAXIM_API_KEY"]
log_repository_id = os.environ["LOG_REPO_ID"]

# Set up logger
logger_config = LoggerConfig(id=log_repository_id)
logger = Logger(config=logger_config, api_key=maxim_api_key, base_url=base_url)

# Set up session
sessionId = str(uuid4())
session_config = SessionConfig(id=sessionId)
session = logger.session(session_config)

# Set up trace
traceId = str(uuid4())
trace_config = TraceConfig(id=traceId)
trace = session.trace(trace_config)

In [20]:
from langchain_openai import ChatOpenAI
from maxim.logger.langchain.tracer import MaximLangchainTracer
from callback_tracer import CallbackTracer

langchain_tracer1 = CallbackTracer(logger, trace, "company_research_agent")
langchain_tracer2 = CallbackTracer(logger, trace,  "website_analysis_agent")
langchain_tracer3 = CallbackTracer(logger, trace,  "content_strategy_agent")
langchain_tracer4 = CallbackTracer(logger, trace, "sales_pitch_agent")
llm1=ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0.7, callbacks=[langchain_tracer1])
llm2=ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0.7, callbacks=[langchain_tracer2])
llm3=ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0.7, callbacks=[langchain_tracer3])
llm4=ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0.7, callbacks=[langchain_tracer4])
# llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0.7, callbacks=[tracer], metadata={"agent": "company_research_agent"})


In [21]:
test = []
cnt = 0
def __step_logging_callback(step_output):
    # This function will be called after each step of the agent's execution
    # print(f"[DEBUG] STEP OUTPUT: {step_output}")
    try:
        
        if isinstance(step_output[0], tuple) and len(step_output[0]) == 2:
            action, observation = step_output[0]
            # if isinstance(action, dict) and "tool" in action and "tool_input" in action and "log" in action:
            #     print(f"[DEBUG] Tool: {action['tool']}")
            # else:
            #     raise Exception("Go to except")
            print(f"[DEBUG] Tool: {action.tool}")
        # if ("tool" in step_output[0]):
        #     print("FOUND")
    except Exception as e:
        print(f"[ERROR]: {e}")
        print(f"[DEBUG] FROM STEPP: {step_output}")

In [22]:
from crewai_tools import FileReadTool, ScrapeWebsiteTool
from langchain_community.tools import DuckDuckGoSearchRun

# Creating search tool
search_tool = DuckDuckGoSearchRun()
# Creating web scraping tool
scrape_tool = ScrapeWebsiteTool()

In [25]:
from crewai import Crew, Task, Agent

# Define the Company Research Agent
company_research_agent = Agent(
    role="Company Research Agent",
    goal="Gather detailed information about the target company, including their business, products, services, and market position.",
    backstory=(
        "Tasked with collecting comprehensive data about potential clients to inform the sales pitch."
    ),
    allow_delegation=True,
    verbose=True,
    llm=llm1,
    step_callback=__step_logging_callback
)

# Define the task for Company Research Agent
company_research_task = Task(
    description=(
        "Collect comprehensive data about {company_name} from their website ({company_website}) and other sources. "
        "Include details about their business, products, services, and market position."
    ),
    expected_output=(
        "Detailed report on {company_name}'s business, products, services, and market position."
    ),
    agent=company_research_agent,
    tools=[search_tool, scrape_tool]
)

In [26]:
# Define the Website Analysis Agent
website_analysis_agent = Agent(
    role="Website Analysis Agent",
    goal="Analyze the target company's website for areas of improvement, including design, performance, and user experience.",
    backstory=(
        "Focuses on identifying specific issues and opportunities for enhancement on the client's website."
    ),
    allow_delegation=True,
    verbose=True,
    llm=llm2,
    step_callback=__step_logging_callback
)

# Define the task for Website Analysis Agent
website_analysis_task = Task(
    description=(
        "Analyze {company_name}'s website ({company_website}) to identify areas of improvement in design, performance, and user experience."
    ),
    expected_output=(
        "Detailed analysis report highlighting areas of improvement for {company_name}'s website, including design, performance, and user experience."
    ),
    agent=website_analysis_agent,
    tools=[scrape_tool, search_tool]
)

In [27]:
# Define the Content Strategy Agent
content_strategy_agent = Agent(
    role="Content Strategy Agent",
    goal="Develop content strategies and identify how improved content can drive engagement and conversions.",
    backstory=(
        "Specializes in creating a compelling narrative and strategy for content improvements on the client's website."
    ),
    allow_delegation=True,
    verbose=True,
    llm=llm3,
    step_callback=__step_logging_callback
)

# Define the task for Content Strategy Agent
content_strategy_task = Task(
    description=(
        "Develop a content strategy for {company_name}'s website ({company_website}) to enhance user engagement and conversion rates."
    ),
    expected_output=(
        "Comprehensive content strategy for {company_name}'s website, including recommendations for content improvements."
    ),
    agent=content_strategy_agent,
    tools=[search_tool, scrape_tool]
)

In [28]:
# Define the Sales Pitch Agent
sales_pitch_agent = Agent(
    role="Sales Pitch Agent",
    goal="Create a personalized and persuasive sales pitch to convince the target company to have their website modified by 'Maxim AI'.",
    backstory=(
        "Combines research, analysis, and content strategy to craft a compelling sales pitch."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm4,
    step_callback=__step_logging_callback
)

# Define the task for Sales Pitch Agent
sales_pitch_task = Task(
    description=(
        "Combine the research, analysis, and content strategy to create a personalized and persuasive sales pitch for {company_name} "
        "to convince them to have their website modified by 'Maxim AI'."
    ),
    expected_output=(
        "Customized sales pitch for {company_name}, highlighting the benefits of having their website modified by 'Maxim AI'."
    ),
    agent=sales_pitch_agent,
    tools=[search_tool, scrape_tool]
)

In [29]:
# Instantiate the Crew
crew = Crew(
    agents=[
        company_research_agent, 
        website_analysis_agent,
        content_strategy_agent,
        sales_pitch_agent
    ],
    tasks=[
        company_research_task, 
        website_analysis_task,
        content_strategy_task,
        sales_pitch_task
    ],
    verbose=2,
    memory=True,
    cache=True
)

2024-09-19 23:26:53,611 - 8220457664 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [30]:
inputs = {
  "company_name": "CrewAI",
  "company_website": "https://crewai.com"
}

# Execute the kickoff
result = crew.kickoff(inputs=inputs)

 [2024-09-19 23:26:53][DEBUG]: == Working Agent: Company Research Agent
 [2024-09-19 23:26:53][INFO]: == Starting Task: Collect comprehensive data about CrewAI from their website (https://crewai.com) and other sources. Include details about their business, products, services, and market position.


> Entering new CrewAgentExecutor chain...
[DEBUG] Messages: [[HumanMessage(content='You are Company Research Agent. Tasked with collecting comprehensive data about potential clients to inform the sales pitch.\nYour personal goal is: Gather detailed information about the target company, including their business, products, services, and market position.\nYou ONLY have access to the following tools, and should NEVER make up tools that are not listed here:\n\nduckduckgo_search - A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.\nRead website content(**kwargs: Any) -> Any - Read website content(website_url: \'str

In [31]:
# print(test)
# trace.event(str(uuid4()), "Timed Test Event", {})
# trace.end()
# logger.cleanup()

In [32]:
print(result)
trace.end()
logger.cleanup()

Dear CrewAI Team,

I hope this message finds you well. After a thorough analysis of your website content, I have identified key areas where Maxim AI's modifications can significantly enhance your website's performance, user experience, and overall engagement.

1. Design:
Our team at Maxim AI can implement a redesign that will declutter your website layout, making it easier for users to navigate and reducing overwhelm. By restructuring repetitive sections, optimizing the color scheme for better readability, and enhancing whitespace and alignment, we can create a more visually appealing and user-friendly interface.

2. Performance:
Maxim AI specializes in optimizing websites for faster loading times. We will address any sections that may be slowing down performance, optimize images and videos for quicker loading speeds, and implement lazy loading for non-essential elements to enhance overall performance. With our expertise, we can ensure optimal performance across different deployment op